# Data pre-processing for Random Forest model

In this notebook we prepare the data that is going to be ingested by the random forest model. A total of 6 features are going to be used: HRV_norm, IR_108, channel differences (WV_062-IR_108) and their textures. The IR_108 contains information about the cloud top height, the channel differences contain information about the water content of the cloud and the HRV provides information about the structure of the cloud. We use the texture of these variables to introduce some information about the spatial structure in this pixel-based retrieval.

In [1]:
import glob
import os
import pyart
import numpy as np
import pandas as pd
from copy import deepcopy


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# suppress anoying iypthon warnings. Not ideal since we suppress also potentially relevant warnings
import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/envs/exts-ml2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Some global variables

In [3]:
fbasepath_input = '/data/pyrad_products/MSG_ML/'
fbasepath_output = '/data/ml_course/05_Capstone_project/rf_data/'
variables = ['HRV', 'HRV_norm', 'HRV_text', 'HRV_norm_text', 'IR_108', 'IR_108_text', 'WV_062-IR_108', 'WV_062-IR_108_text', 'POH90']

years = ['2018', '2019', '2020']
months = ['04', '05', '06', '07', '08', '09']

features = ['HRV_norm', 'HRV_norm_text', 'IR_108', 'IR_108_text', 'WV_062-IR_108', 'WV_062-IR_108_text']
target = 'POH90'
nfeatures = len(features)

## Auxiliary functions

In [4]:
# Function to read original dataset
# data is stored as (nz, ny, nx), we return (nx, ny)
def read_nc(fname):
    sat_grid = pyart.io.read_grid(fname)
    for field_name in sat_grid.fields.keys():
        data = np.transpose(np.squeeze(sat_grid.fields[field_name]['data']))
    return data        

In [5]:
# Filter data where IR temperature too high or channel differences too negative
def filter_data(X, y, ir_thr=240, diff_thr=-50):
    # Find row indexes of elements to delete
    ind = []
    ind_ir = np.where(X[:, 2] > ir_thr)
    if len(ind_ir[0]) > 0:
        ind.extend(ind_ir[0])
    ind_diff = np.where(X[:, 4] < diff_thr)
    if len(ind_diff[0]) > 0:
        ind.extend(ind_diff[0])
    if len(ind) == 0:
        return X, y
    
    # delete from feature matrix
    X2 = np.delete(X, ind, axis=0)
    
    # delete from target matrix
    y2 = np.delete(y, ind, axis=0)
    
    return X2, y2

## Data processing

In [6]:
# Check number of files for each variable
for var in variables:
    flist = glob.glob(fbasepath_input+'*/NETCDF/'+var+'/*.nc')
    print(var, len(flist))

HRV 4353
HRV_norm 4353
HRV_text 4353
HRV_norm_text 4353
IR_108 4355
IR_108_text 4355
WV_062-IR_108 4355
WV_062-IR_108_text 4355
POH90 4355


In [7]:
## This function will identify the missing HRV files
#flist = glob.glob(fbasepath+'*/NETCDF/'+variables[-1]+'/*nc')
#flist.sort()
#for fname in flist:
#    bfile = os.path.basename(fname)
#    dt_str = bfile[0:14]
#    print(dt_str, end="\r", flush=True)
#    for var in vars:
#        flist_aux = glob.glob(fbasepath+'*/NETCDF/'+var+'/'+dt_str+'*.nc')
#        if len(flist_aux) > 0:
#            pass
#            # print(flist_aux[0])
#        else:
#            print(dt_str, var)

We keep only data where potentially hail could be located. That means clouds that have a high top (low IR_108 values) and small channel differences (high water content). The features are stored in a feature matrix with 6 columns. A file per month is generated.

In [8]:
for year in years:
    for month in months:
        # Get list of files and data size
        flist = glob.glob(fbasepath_input+'*/NETCDF/'+features[0]+'/'+year+month+'*.nc')
        if len(flist) == 0:
            continue
        flist.sort()
        img_size = read_nc(flist[0]).shape
        data_size = img_size[0]*img_size[1]
        
        X = None
        for fname in flist:
            # Get time step
            bfile = os.path.basename(fname)
            dt_str = bfile[0:14]
            print(dt_str, end="\r", flush=True)
            
            # Read all files corresponding to a time step
            # Put them in features and target matrices
            X_aux = np.empty((data_size, nfeatures), dtype=np.float32)
            for i, feature in enumerate(features):
                flist_aux = glob.glob(fbasepath_input+'*/NETCDF/'+feature+'/'+dt_str+'*.nc')
                data = read_nc(flist_aux[0]).flatten()
                X_aux[:, i] = data
               
            flist_aux = glob.glob(fbasepath_input+'*/NETCDF/'+target+'/'+dt_str+'*.nc')
            y_aux = np.transpose(read_nc(flist_aux[0]).flatten())
            
            # Filter data
            X_aux, y_aux = filter_data(X_aux, y_aux)
            
            # Put all data together
            if X is None:
                X = X_aux
                y = y_aux
            else:
                X = np.concatenate((X, X_aux), axis=0)
                y = np.concatenate((y, y_aux), axis=0)
                
        # Save data into a .npz file
        np.savez(fbasepath_output+year+month+'_data.npz', features=X, targets=y)